# Import

In [1]:
import sys

root_dir = '../../'
if root_dir not in sys.path:
    sys.path.append(root_dir)

import torch
from torch import nn, optim
import numpy as np
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix

from modules import losses, models, samplers, regularizers, evaluators, trainers, datasets, distributions

# DataSet

In [4]:
dataset = datasets.ML100k()
n_user = dataset.n_user
n_item = dataset.n_item
train_set, test_set = dataset.implicit_feedback_data_old(tail_threshold=20)

# device setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
train_set = torch.LongTensor(train_set).to(device)
test_set = torch.LongTensor(test_set).to(device)

# Evaluator

In [6]:
# k
ks = [10, 50]

# Accuracy
recall_evaluator = evaluators.RecallEvaluator(test_set, ks)

# Coverage
coverage_evaluator = evaluators.CoverageEvaluator(test_set, ks)

# Diversity
diversity_evaluator = evaluators.DiversityEvaluator(test_set, item_feature_set, ks, emb_sim=False)

# Hubness
hubness_evaluator = evaluators.HubnessEvaluator(test_set, ks)

# Unpopularity
unpopularity_evaluator = evaluators.UnpopularityEvaluator(test_set, ks)

# F1-Score (Recall-Unpopularity)
f1_score_evaluator = evaluators.F1ScoreEvaluator(test_set, ks)

# Sampler

In [7]:
sampler = samplers.BaseSampler(train_set, n_user, n_item, device=device, strict_negative=False)

# Model

In [9]:
# Hyperparameters
lr = 1e-3
n_dim = 10
n_batch = 256
n_epoch = 30
valid_per_epoch = 10

# distributiuons
dist1 = distributions.Empirical()
dist2 = distributions.Gaussian()
dist3 = distributions.Gamma()

# models
model0 = models.CollaborativeMetricLearning(n_user, n_item, n_dim).to(device)
model1 = models.MutualProximityCML(n_user, n_item, dist1, n_dim).to(device)
# Empericalと条件を揃えるために S=n_item,n_user とする
model2 = models.MutualProximityCML(n_user, n_item, dist2, n_dim).to(device)
model3 = models.MutualProximityCML(n_user, n_item, dist3, n_dim).to(device)

# learning late optimizer
optimizer0 = optim.Adam(model0.parameters(), lr=lr)
optimizer1 = optim.Adam(model1.parameters(), lr=lr)
optimizer2 = optim.Adam(model2.parameters(), lr=lr)
optimizer3 = optim.Adam(model3.parameters(), lr=lr)

# loss function
criterion = losses.SumTripletLoss(margin=1).to(device)

# trainer
trainer0 = trainers.BaseTrainer(model0, optimizer0, criterion, sampler)
trainer1 = trainers.BaseTrainer(model1, optimizer1, criterion, sampler)
trainer2 = trainers.BaseTrainer(model2, optimizer2, criterion, sampler)
trainer3 = trainers.BaseTrainer(model3, optimizer3, criterion, sampler)

# Accuracy

In [11]:
# train0
trainer0.fit(n_batch, n_epoch, recall_evaluator, valid_per_epoch)

100%|███████████████████████████████████████████████████████████████| 942/942 [00:01<00:00, 628.55it/s]


In [12]:
# train3
trainer3.fit(n_batch, n_epoch, recall_evaluator, valid_per_epoch)

100%|████████████████████████████████████████████████████████████████| 942/942 [00:16<00:00, 57.84it/s]


In [13]:
# CML
trainer0.valid_scores

,Recall@10,Recall@50,epoch,losses
0,0.006542,0.035115,0,NaN
0,0.023819,0.178511,10,0.475693
0,0.121679,0.398849,20,0.303139
0,0.148057,0.472039,30,0.254808


In [14]:
# MPCML
trainer3.valid_scores

,Recall@10,Recall@50,epoch,losses
0,0.006465,0.033527,0,NaN
0,0.015673,0.085314,10,0.447292
0,0.060472,0.237378,20,0.309986
0,0.087165,0.306482,30,0.260155


# Coverage

In [15]:
# train0
trainer0.fit(n_batch, n_epoch, coverage_evaluator, valid_per_epoch)

epoch12 avg_loss:0.238:   0%|                                                  | 0/256 [00:00<?, ?it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

epoch27 avg_loss:0.228:   0%|                                                  | 0/256 [00:00<?, ?it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

100%|██████████████████████████████████████████████████████████████| 942/942 [00:00<00:00, 4876.87it/s]


In [16]:
# train3
trainer3.fit(n_batch, n_epoch, coverage_evaluator, valid_per_epoch)

100%|████████████████████████████████████████████████████████████████| 942/942 [00:15<00:00, 62.09it/s]


In [17]:
# CML
trainer0.valid_scores

,coverage@10,coverage@50,epoch,losses
0,0.587422,0.818936,0,NaN
0,0.559779,0.790601,10,0.241700
0,0.550795,0.774706,20,0.232954
0,0.552177,0.778853,30,0.230889


In [18]:
# MPCML
trainer3.valid_scores

,coverage@10,coverage@50,epoch,losses
0,0.724948,0.938493,0,NaN
0,0.702833,0.926054,10,0.245349
0,0.705598,0.923290,20,0.237001
0,0.696614,0.917761,30,0.236676


# Hubness 

In [18]:
# train0
trainer0.fit(n_batch, n_epoch, hubness_evaluator, valid_per_epoch)

100%|████████████████████████████████████████████████████████████████████| 943/943 [00:06<00:00, 155.59it/s]


In [19]:
# train3
trainer3.fit(n_batch, n_epoch, hubness_evaluator, valid_per_epoch)

100%|█████████████████████████████████████████████████████████████████████| 943/943 [01:26<00:00, 10.90it/s]


In [20]:
# CML
trainer0.valid_scores

,hubness@10,hubness@50,hubness@100,epoch,losses
0,2.993997,2.472236,1.904482,0,NaN
0,2.994896,2.433877,1.835709,10,0.174507
0,2.954879,2.468611,1.895725,20,0.172921
0,3.337296,2.345126,1.821007,30,0.173450
0,3.213979,2.637973,1.912664,40,0.174304
0,3.147580,2.529649,1.866664,50,0.173165


In [21]:
# MPCML
trainer3.valid_scores

,hubness@10,hubness@50,hubness@100,epoch,losses
0,1.155360,0.842149,0.663550,0,NaN
0,1.135682,0.854684,0.677147,10,0.172490
0,1.099152,0.830545,0.675085,20,0.173634
0,1.080685,0.862235,0.653784,30,0.174280
0,1.185578,0.862683,0.672255,40,0.174261
0,1.082539,0.867734,0.663411,50,0.173430


# Unpopularity

In [19]:
# train0
trainer0.fit(n_batch, n_epoch, unpopularity_evaluator, valid_per_epoch)

100%|███████████████████████████████████████████████████████████████| 942/942 [00:01<00:00, 604.25it/s]


In [20]:
# train3
trainer3.fit(n_batch, n_epoch, unpopularity_evaluator, valid_per_epoch)

100%|████████████████████████████████████████████████████████████████| 942/942 [00:16<00:00, 57.54it/s]


In [21]:
# CML
trainer0.valid_scores

,Unpopularity@10,Unpopularity@50,epoch,losses
0,0.000679,0.006978,0,NaN
0,0.000679,0.007146,10,0.228416
0,0.000701,0.007221,20,0.228856
0,0.000710,0.007366,30,0.226355


In [22]:
# MPCML
trainer3.valid_scores

,Unpopularity@10,Unpopularity@50,epoch,losses
0,0.003101,0.020848,0,NaN
0,0.003195,0.021103,10,0.230344
0,0.003181,0.021262,20,0.231484
0,0.003138,0.021086,30,0.230748


In [6]:
# train0
trainer0.fit(n_batch, n_epoch, f1_score_evaluator, valid_per_epoch)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 943/943 [00:16<00:00, 57.25it/s]


In [7]:
# train3
trainer3.fit(n_batch, n_epoch, f1_score_evaluator, valid_per_epoch)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 943/943 [01:47<00:00,  8.80it/s]


In [8]:
# CML
trainer0.valid_scores

,F1-Score@10,F1-Score@50,F1-Score@100,epoch,losses
0,0.000831,0.014719,0.039676,0,NaN
0,0.000027,0.001278,0.008496,10,0.223168
0,0.000205,0.005538,0.021902,20,0.185586
0,0.000284,0.006709,0.024035,30,0.180622
0,0.000310,0.006750,0.024082,40,0.177439
0,0.000304,0.006892,0.024555,50,0.175953


In [9]:
# MPCML
trainer3.valid_scores

,F1-Score@10,F1-Score@50,F1-Score@100,epoch,losses
0,0.000867,0.014002,0.038383,0,NaN
0,0.001985,0.022243,0.057425,10,0.226947
0,0.002767,0.027332,0.067365,20,0.187260
0,0.002999,0.029062,0.068615,30,0.179385
0,0.003123,0.029503,0.069295,40,0.176829
0,0.003081,0.029455,0.069231,50,0.177244
